In [1]:
import os
%pwd

'd:\\personal_files\\python\\end_to_End_nlp_projects\\1_text_summarization\\__github__\\text-summarization\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\personal_files\\python\\end_to_End_nlp_projects\\1_text_summarization\\__github__\\text-summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textsummarization import constants
from textsummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path = constants.CONFIG_FILE_PATH, params_file_path = constants.PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_roots])
    
    def get_data_transformation_config(self, ):
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir, data_path=config.data_path,tokenizer_name=config.tokenizer_name
            )
        return data_transformation_config

In [8]:
import os
from textsummarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):

        input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], max_length = 128, truncation = True)

        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
        }
    
    def convert(self, ):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-07-08 11:52:38,668: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-08 11:52:38,668: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-08 11:52:38,668: INFO: common: Created directory at: artifacts]
[2023-07-08 11:52:38,678: INFO: common: Created directory at: artifacts/data_transformation]


d:\personal_files\python\end_to_End_nlp_projects\1_text_summarization\__github__\text-summarization\myenv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\n.anurag\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\personal_